In [6]:
import pandas as pd
import csv
import pickle
import numpy as np
from IPython.display import display
import multiprocessing
from multiprocessing import Pool
import math
from tqdm import tqdm
import time
import traceback
from sklearn.externals import joblib
import xgboost as xgb
import json
import time
from tqdm import tqdm

In [7]:
event = 'nyc'
start_hour = 730

In [8]:
#path = "/Users/jay/MSC_WSBDA/MSc_Thesis/Msc_project/Data/"
path = '/Users/syamantak/JayateeB/new_files/data/'
#model = joblib.load(path+"xgb_model_GIVENCHY (2).dat")
model = joblib.load(path+event+"_xgb_model_"+str(start_hour)+"_hrs.dat")

In [9]:
def load_pickle_file(pickled_file):
    print(f'Loading data file from {pickled_file}')
    infile = open(pickled_file,'rb')
    unpickled_file = pickle.load(infile)
    print(f'Loaded {len(unpickled_file)} entries')
    infile.close()
    return unpickled_file
          
    
def save_pickle_file(path, data):
    print('Dumping data to path {}'.format(path))
    with open(path, 'wb') as file:
        pickle.dump(data, file)
    print('Finished dumping data to path {}'.format(path))


def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)


def safe_division(x, y):
    if y == 0:
        return 0
    else:
        return x/y



In [12]:
initial_features = load_pickle_file(path+event+str(start_hour)+'_hrs_data.pkl')
initial_dataset = load_pickle_file(path+event+'_network_simulation_'+str(start_hour)+'_hrs.pkl')
#users = load_pickle_file(path+event+"_users.dat")
#users.reset_index(drop =True , inplace =True)
#initial_features = load_pickle_file(path+'sample_hrs_data.pkl')
#initial_dataset = load_pickle_file(path+'nyc_network_simulation_sample_hrs.pkl')

Loading data file from /Users/syamantak/JayateeB/new_files/data/nyc7_hrs_data.pkl
Loaded 1832451 entries
Loading data file from /Users/syamantak/JayateeB/new_files/data/nyc_network_simulation_7_hrs.pkl
Loaded 1832451 entries


In [14]:
users = load_pickle_file(path+'nyc/'+'nyc_users.dat')

Loading data file from /Users/syamantak/JayateeB/new_files/data/nyc/nyc_users.dat
Loaded 135912 entries


In [15]:
in_degree = list(initial_dataset.friends_count)
out_degree = list(initial_dataset.followers_count)
degree = in_degree + out_degree

In [29]:
def process_data(source_id,target_id,features,network_simulation,current_time):
    
    if isinstance(network_simulation.loc[target_id,'source_candidates'],list) and len(network_simulation.loc[target_id,'source_candidates']) > 0 :
        
        source_candidates = sorted(network_simulation.loc[target_id,'source_candidates'])
        nf = network_simulation.loc[source_candidates]
        
        sources = nf[nf['time_lapsed'] <= current_time].index.tolist()
        
        if len(sources) > 0:
        
            first_source_index = source_candidates[0]
            first_source_row = network_simulation.loc[first_source_index]
            first_source_seed_row = network_simulation.loc[first_source_row['seed_index']]

            sources_dataframe = network_simulation.loc[sources]
            degreeList = list(degree[i] for i in sources)
            inDegreeList = list(in_degree[i] for i in sources)
            outDegreeList = list(out_degree[i] for i in sources)
            degreeList = list(network_simulation.loc[i, 'followers_count'] + network_simulation.loc[i, 'friends_count']  for i in sources)
            timeList = [current_time - network_simulation.loc[x,'time_lapsed'] for x in sources]


            last_source_index = sources[-1]    
            try:
                last_source_row = network_simulation.loc[last_source_index]
                last_source_seed_row = network_simulation.loc[last_source_row['seed_index']]
            except:
                print(f"target_index:{target_id}, last_source_row['seed_index'] : {last_source_row['seed_index']}")
                print(f"last_source_index:{last_source_index}")


            #Extraction
            #Columns which are added for simulation, but they are not used as features for model prediction

            user_row = network_simulation.loc[target_id]


            # UsM: User metadata                    

            features.loc[target_id,'UsM_deltaDays0'] = first_source_row.user_created_days
            features.loc[target_id,'UsM_statusesCount0'] = first_source_row.statuses_count
            features.loc[target_id,'UsM_followersCount0'] = first_source_row.followers_count
            features.loc[target_id,'UsM_favouritesCount0'] = first_source_row.favourites_count
            features.loc[target_id,'UsM_friendsCount0'] = first_source_row.friends_count
            features.loc[target_id,'UsM_listedCount0'] = first_source_row.listed_count
            features.loc[target_id,'UsM_normalizedUserStatusesCount0'] = first_source_row.normalized_statuses_count
            features.loc[target_id,'UsM_normalizedUserFollowersCount0'] = first_source_row.normalized_followers_count
            features.loc[target_id,'UsM_normalizedUserFavouritesCount0'] = first_source_row.normalized_favourites_count
            features.loc[target_id,'UsM_normalizedUserListedCount0'] = first_source_row.normalized_listed_count
            features.loc[target_id,'UsM_normalizedUserFriendsCount0'] = first_source_row.normalized_friends_count
            features.loc[target_id,'UsM_deltaDays-1'] = last_source_row.user_created_days
            features.loc[target_id,'UsM_statusesCount-1'] = last_source_row.statuses_count
            features.loc[target_id,'UsM_followersCount-1'] = last_source_row.followers_count
            features.loc[target_id,'UsM_favouritesCount-1'] = last_source_row.favourites_count
            features.loc[target_id,'UsM_friendsCount-1'] = last_source_row.friends_count
            features.loc[target_id,'UsM_listedCount-1'] = last_source_row.listed_count
            features.loc[target_id,'UsM_normalizedUserStatusesCount-1'] = last_source_row.normalized_statuses_count
            features.loc[target_id,'UsM_normalizedUserFollowersCount-1'] = last_source_row.normalized_followers_count
            features.loc[target_id,'UsM_normalizedUserFavouritesCount-1'] = last_source_row.normalized_favourites_count
            features.loc[target_id,'UsM_normalizedUserListedCount-1'] = last_source_row.normalized_listed_count
            features.loc[target_id,'UsM_normalizedUserFriendsCount-1'] = last_source_row.normalized_friends_count
            # TwM: Tweet metadata
            features.loc[target_id,'TwM_t0'] = round(timeList[0], 1)
            features.loc[target_id,'TwM_tSeed0'] = round(current_time - first_source_seed_row['time_lapsed'], 1)
            features.loc[target_id,'TwM_t-1'] = round(timeList[-1], 1)
            features.loc[target_id,'TwM_tSeed-1'] = round(current_time - last_source_seed_row['time_lapsed'], 1)
            features.loc[target_id,'TwM_tCurrent'] = current_time
            # Nw: Network
            features.loc[target_id,'Nw_degree'] = degree[target_id]
            features.loc[target_id,'Nw_inDegree'] = in_degree[target_id]
            features.loc[target_id,'Nw_outDegree'] = out_degree[target_id]
            features.loc[target_id,'Nw_degree0'] = degree[first_source_index]
            features.loc[target_id,'Nw_inDegree0'] = in_degree[first_source_index]
            features.loc[target_id,'Nw_outDegree0'] = out_degree[first_source_index]
            features.loc[target_id,'Nw_degree-1'] = degree[last_source_index]
            features.loc[target_id,'Nw_inDegree-1'] = in_degree[last_source_index]
            features.loc[target_id,'Nw_outDegree-1'] = out_degree[last_source_index]
            features.loc[target_id,'Nw_degreeSeed0'] = degree[int(first_source_row['seed_index'])]
            features.loc[target_id,'Nw_inDegreeSeed0'] = in_degree[int(first_source_row['seed_index'])]
            features.loc[target_id,'Nw_outDegreeSeed0'] = out_degree[int(first_source_row['seed_index'])]
            features.loc[target_id,'Nw_degreeSeed-1'] = degree[int(last_source_row['seed_index'])]
            features.loc[target_id,'Nw_inDegreeSeed-1'] = in_degree[int(last_source_row['seed_index'])]
            features.loc[target_id,'Nw_outDegreeSeed-1'] = out_degree[int(last_source_row['seed_index'])]
            # SNw: Spreading Network
            features.loc[target_id,'SNw_nFriendsInfected'] = len(sources)
            features.loc[target_id,'SNw_friendsInfectedRatio'] = safe_division(len(sources), user_row['friends_count'])
            features.loc[target_id,'SNw_generation0'] = first_source_row['generation']
            features.loc[target_id,'SNw_generation-1'] = last_source_row['generation']
            features.loc[target_id,'SNw_timeSinceSeed0'] = first_source_row['time_since_seed']
            features.loc[target_id,'SNw_timeSinceSeed-1'] = last_source_row['time_since_seed']

            infected_dataframe = network_simulation[network_simulation.time_lapsed <= current_time]
            total_nodes_infected = infected_dataframe.shape[0]
            total_in_degree = sum(infected_dataframe.friends_count)
            total_out_degree = sum(infected_dataframe.followers_count)

            features.loc[target_id,'SNw_totalNodesInfected'] = total_nodes_infected
            features.loc[target_id,'SNw_nodeInfectedCentrality'] = len(sources)/total_nodes_infected
            features.loc[target_id,'SNw_totalInDegree'] = total_in_degree
            features.loc[target_id,'SNw_totalOutDegree'] = total_out_degree
            features.loc[target_id,'SNw_inDegreeCentrality'] = in_degree[target_id]/total_in_degree
            features.loc[target_id,'SNw_inDegreeCentrality0'] = in_degree[first_source_index]/total_in_degree
            features.loc[target_id,'SNw_inDegreeCentrality-1'] = in_degree[last_source_index]/total_in_degree
            features.loc[target_id,'SNw_outDegreeCentrality'] = out_degree[target_id]/total_out_degree
            features.loc[target_id,'SNw_outDegreeCentrality0'] = out_degree[first_source_index]/total_out_degree
            features.loc[target_id,'SNw_outDegreeCentrality-1'] = out_degree[last_source_index]/total_out_degree
            features.loc[target_id,'SNw_inDegreeCentralitySeed0'] = in_degree[int(first_source_row['seed_index'])]/total_in_degree
            features.loc[target_id,'SNw_outDegreeCentralitySeed0'] = out_degree[int(first_source_row['seed_index'])]/total_out_degree
            features.loc[target_id,'SNw_inDegreeCentralitySeed-1'] = in_degree[int(last_source_row['seed_index'])]/total_in_degree
            features.loc[target_id,'SNw_outDegreeCentralitySeed-1'] = out_degree[int(last_source_row['seed_index'])]/total_out_degree
            # Stat: Statistical
            features.loc[target_id,'Stat_average_kOut'] = round(mean(degreeList), 1)
            features.loc[target_id,'Stat_average_t'] = round(mean(timeList), 1)
            features.loc[target_id,'Stat_average_deltaDays'] = sources_dataframe.user_created_days.mean()
            features.loc[target_id,'Stat_average_statusesCount'] = sources_dataframe.statuses_count.mean()
            features.loc[target_id,'Stat_average_followersCount'] = sources_dataframe.followers_count.mean()
            features.loc[target_id,'Stat_average_favouritesCount'] = sources_dataframe.favourites_count.mean()
            features.loc[target_id,'Stat_average_friendsCount'] = sources_dataframe.friends_count.mean()
            features.loc[target_id,'Stat_average_listedCount'] = sources_dataframe.listed_count.mean()
            features.loc[target_id,'Stat_average_normalizedUserStatusesCount'] = sources_dataframe.normalized_statuses_count.mean()
            features.loc[target_id,'Stat_average_normalizedUserFollowersCount'] = sources_dataframe.normalized_followers_count.mean()
            features.loc[target_id,'Stat_average_normalizedUserFavouritesCount'] = sources_dataframe.normalized_favourites_count.mean()
            features.loc[target_id,'Stat_average_normalizedUserListedCount'] = sources_dataframe.normalized_listed_count.mean()
            features.loc[target_id,'Stat_average_normalizedUserFriendsCount'] = sources_dataframe.normalized_friends_count.mean()
            features.loc[target_id,'Stat_max_kOut'] = max(degreeList)
            features.loc[target_id,'Stat_min_kOut'] = min(degreeList)
    #processed_dataframe = pd.DataFrame(features)
    return features



In [38]:
def simulation(features,network_simulation,current_time):
    infected_users_indices = network_simulation[network_simulation['time_lapsed'].isnull() == False].index.tolist()
    #print(f"infected_users_indices:{infected_users_indices}")
    for i in tqdm(infected_users_indices, total=len(infected_users_indices)):
        if isinstance(network_simulation.loc[i,'followers_list'],str):
            flrs_list = list(map(lambda x: int(x), json.loads(network_simulation.loc[i].followers_list)))
            #print(f"flrs_list:{flrs_list}")
            common_x = list(set(flrs_list).intersection(set(id_set)))
            df = network_simulation[network_simulation['id'].isin(common_x)]
            followers_indices = df.index.tolist()                      
            #uninfected_followers_indices = [y for y in followers_indices if np.isnan(network_simulation.loc[y, 'time_lapsed']) == True]
            df_flrs = network_simulation.loc[followers_indices]
            uninfected_followers_indices = df_flrs[df_flrs['time_lapsed'].isnull() == True].index.tolist() 
            #print(f"i:{i},followers_indices:{followers_indices},uninfected_followers_indices:{uninfected_followers_indices}")
            if len(uninfected_followers_indices) > 0:
                #print(f"uninfected followers are there")
                for j in uninfected_followers_indices:
                    #print(f"j:{j}")
                    source_index = i
                    target_index = j
                    processed_dataframe = process_data(source_index,target_index,features,network_simulation,current_time)
                    #print(f"len processed_dataframe: {processed_dataframe[0,:]}")
                    
                    valid_row = processed_dataframe.loc[[target_index]]
                    #print(f"(valid_row.columns):{list(valid_row.columns.values)}")
                    
                    #for i in range(valid_row.columns):
                        #print(f"valid_row:{i}")
                    #valid = valid_row.drop(['user_id','infected_status','infection_time','followers_list','Nw_inDegree','Nw_outDegree'],axis=1)
                    valid = valid_row.drop(['user_id','infected_status','infection_time','followers_list'],axis=1)
                    #print(f"list(valid.columns):{list(valid.columns)}")
                    valid = valid.astype('float64')

                    columns = list(valid.columns)
                    X = valid[columns]
                    pre_data = xgb.DMatrix(X)
                    infec = model.predict(pre_data)
                    #print(f"infec:{infec}")
                    if infec > 0.6:
                        print("Infected")
                        network_simulation.loc[target_index,'time_lapsed'] = current_time
                        network_simulation.loc[target_index,'source_index'] = source_index
                        network_simulation.loc[target_index,'seed_index'] = network_simulation.loc[source_index,'seed_index']
                        network_simulation.loc[target_index,'generation'] = network_simulation.loc[source_index,'generation'] + 1
                        seed_index = network_simulation.loc[target_index,'seed_index']
                        network_simulation.loc[target_index,'time_since_seed'] = current_time - network_simulation.loc[seed_index,'time_lapsed']
                        followers_of_node = network_simulation.loc[target_index,'followers_list'] 
                        if isinstance(followers_of_node,list):
                            for f in followers_of_node:
                                if np.isnan(network_simulation[network_simulation['id'] == f]['time_lapsed'].values):
                                    follower_index = network_simulation[network_simulation['id'] == f].index.values
                                    try:
                                        list(network_simulation.loc[follower_index,'source_candidates'].values).append(f)
                                    except:
                                        print(f"source_candidates:{network_simulation.loc[follower_index,'source_candidates'].values}")
                                        print(f"f:{f}")

    return network_simulation

In [31]:
total_time_duration = 450
interval = 30
#current_time = 360
current_time = 420
#current_time = 480
#current_time = 540
#current_time = 720




features = initial_features
network_simulation = initial_dataset
id_set = set(network_simulation['id'].tolist())

print("Simulation started")
start_time = time.time()


while current_time < total_time_duration:
    print(f"current_time:{current_time}")
    #network_simulation = simulation(features,initial_dataset,network_simulation,current_time)
    network_simulation = simulation(features,initial_dataset,network_simulation,current_time)
    #print(f"current_time:{current_time}")
    current_time += interval
    
print(f"Simulation finished after {round((time.time() - start_time)/60,2)} minutes")
    



  5%|▍         | 3990/81124 [00:00<00:01, 39898.52it/s]

Simulation started
current_time:420



 92%|█████████▏| 74858/81124 [00:01<00:00, 37143.53it/s]

Infected
Infected



 94%|█████████▍| 76103/81124 [00:14<00:14, 335.07it/s]  

...

Infected



100%|█████████▉| 81116/81124 [14:10:23<00:56,  7.06s/it]

Infected
Infected



100%|██████████| 81124/81124 [14:11:05<00:00,  9.81s/it]

Simulation finished after 851.1 minutes


In [48]:
current_time = 540
network_simulation = simulation(features,network_simulation,current_time)


 92%|█████████▏| 76402/82690 [43:57<12:45:22,  7.30s/it]


 93%|█████████▎| 76717/82690 [1:24:35<5:51:53,  3.53s/it]


 93%|█████████▎| 77025/82690 [2:12:20<9:58:42,  6.34s/it] 


 94%|█████████▎| 77327/82690 [2:53:25<8:32:02,  5.73s/it]


 94%|█████████▍| 77646/82690 [3:38:39<27:40:58, 19.76s/it]


...

 98%|█████████▊| 80776/82690 [11:22:26<9:26:55, 17.77s/it]


 98%|█████████▊| 81101/82690 [12:11:21<4:56:24, 11.19s/it]


 98%|█████████▊| 81371/82690 [13:15:42<7:31:28, 20.54s/it]


 99%|█████████▊| 81639/82690 [14:25:03<9:58:29, 34.17s/it]


100%|██████████| 82690/82690 [15:40:38<00:00,  1.47it/s]  

In [46]:
features.to_csv(path+event+'-features_from_510_minutes.csv')

In [47]:
network_simulation.to_csv(path+event+'_simulation_result_from_510_minutes.csv')

In [33]:
len(network_simulation)

1832451

In [45]:
df_after_7.groupby(['time_lapsed']).count()['id']

time_lapsed
420.0    1675698
450.0        481
480.0         44
510.0         11
Name: id, dtype: int64

In [53]:
len(infected_followers_predicted)

1703274

In [10]:
#Precision Recall:

simulation_comparison = pd.concat([network_simulation['time_lapsed'],users['time_lapsed']],axis = 1)
column_names = ['nw_time_lapsed','usr_time_lapsed']
simulation_comparison.columns = column_names

simulation_comparison['both_infected'] = simulation_comparison.apply(lambda x: 1 if (np.isnan(x['nw_time_lapsed']) == False)
                                                                          & (np.isnan(x['usr_time_lapsed']) == False)
                                                                     else 0,axis = 1)

true_positive = simulation_comparison['both_infected'].sum()
total_predicted_positive = len(simulation_comparison[np.isnan(simulation_comparison['nw_time_lapsed']) == False])
precision = true_positive/total_predicted_positive
total_actual_positive = len(simulation_comparison[np.isnan(simulation_comparison['usr_time_lapsed']) == False])
recall = true_positive/total_actual_positive
f1_score = (2*(precision * recall))/(precision + recall)

# accuracy['simulation_start_time'].append(6)
# accuracy['precision'].append(precision)
# accuracy['recall'].append(recall)

# accuracy['f1_score'].append(f1_score)






In [11]:
accuracy = {
    
    'simulation_start_time':[],
    'precision':[],
    'recall':[],
    'f1_score':[]
    }

accuracy['simulation_start_time'].append(14)
accuracy['precision'].append(precision)
accuracy['recall'].append(recall)

accuracy['f1_score'].append(f1_score)
result = pd.DataFrame(accuracy)
result.to_csv(path+event+'_simulation_accuracy_'+str(start_hour)+'hrs_'+str(start_hour)+'hrs_model.csv')